In [1]:
#importing modules for web-scraping
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager 
import pandas as pd
import time
import math
from collections import ChainMap



#importing modules for the API key 
#requests are already imported 
import json

# Google API Key
from config import gkey


In [2]:
#open the Chrome Driver web browser in order to do web-scraping 
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#Web Scraping 1- need to interact with the job-seeking website first of all


jobs_find_url ="https://www.seek.com.au/jobs/in-All-Australia?keywords=%22data%20%20analyst%22"
browser.visit(jobs_find_url)
jobs_html=browser.html 

The first step is to get the web-page url from the seek jobs website where the job includes "data analyst". The job search for this page is limited to all of Australia


Next, the number of pages that the search included needed to be obtained so that we know how many pages to loop through. It was found that there were 22 listings per page, and thus the numebr of jobs was divided by 22. After this it was noticed that the job advertisements just duplicate. Thus a decision was made to round up the number of pages/22 and then delete duplicate entries later. 

In [4]:
# Retrieve page with the requests module

#it was noticed that the seek page had 22 jobs listed per page. Thus in order to obtain the number of pages, first need to obtain the number of pages:
#Number of jobs listed at the top of the page, the following code obtains this:
job_soup=bs(jobs_html, 'html.parser')
job_number=job_soup.find('span', class_="_3FrNV7v _3PZrylH _2heRYaN").text

number=job_number.strip('jobs found')

number_1=float(number)
print(number_1)

number_pages=math.ceil(number_1/22)
print(number_pages)

time.sleep(2)


220.0
10


The code in the excerpt below saves the job title, location and company for data analyst jobs throughout Australia into lists. This was done through using looping and webscraping. 

In [5]:
#getting the browser html
text_list=[]
job_location=[]
job_title1=[]
job_company=[]
job_classification=[]
x=range(1,number_pages+1)
for n in x:
    seek_url=f"https://www.seek.com.au/jobs/in-All-Australia?keywords=%22data%20%20analyst%22&page={n}"
    browser.visit(seek_url)
    seek_html=browser.html  
    seek_soup=bs(seek_html, 'html.parser')
    all_text=seek_soup.find('div', class_='_3MPUOLE')
    job_title=all_text.find_all('span', class_='_3FrNV7v _2IOW3OW HfVIlOd _2heRYaN E6m4BZb')
    extra_class=all_text.find_all('span', class_="Eadjc1o")
    job_characteristics=all_text.find_all('span', class_="_3FrNV7v _3PZrylH E6m4BZb")

    #converting each of the elements to text 
    
    #modifying the job location element and tidying up this element
    for element in extra_class:
        element1=element.text
        if "location:" in element1:
            element2=element1.lstrip('location: ')
            job_location.append(element2) 
    
    job_title=all_text.find_all('span', class_='_3FrNV7v _2IOW3OW HfVIlOd _2heRYaN E6m4BZb') 
    
    #editing both the job title and job characterstics
    for item in job_title:
        job1=item.text
        job_title1.append(job1)
    
    for company in job_characteristics:
        company1=company.text
        if company1[:3]=='at ':
            company2=company1.lstrip('at ')
            job_company.append(company2) 
    
    for job_class in extra_class:
        classification1=job_class.text
        if "classification:" in classification1:
            classification2=classification1.lstrip('classification: ')
            job_classification.append(classification2) 
    
    time.sleep(3)
            

In [6]:
browser.quit()

The lists were then combined in a pandas dataframe

In [7]:
#zip two lists together 

Job_df = pd.DataFrame(
    {'Job Title': job_title1,
     'Job Location': job_location,
     'Job Company': job_company,
     'Job Industry': job_classification
    })


In [8]:
Job_df.head(20)

,Job Title,Job Location,Job Company,Job Industry
0,Junior Insights Analyst,Sydney,Oneflare Pty Ltd,Sales
1,Graduate Data Analyst & Programmer,Gold Coast,Tebbutt Research,Marketing & Communications
2,Data Analyst,West Gippsland & Latrobe Valley,Hays Information Technology,Government & Defence
3,Data Analyst/Manager,Sydney,Finite IT Recruitment Solutions,Information & Communication Technology
4,Data Analyst,ACT,Hudson - Project Services,Information & Communication Technology
5,Graduate Estate Data Analyst,Melbourne,Ventia,Construction
6,Data Analyst,Sydney,Teachers Mutual Bank,Information & Communication Technology
7,Customer Insights Data Analyst,Brisbane,eraDigital Services,Marketing & Communications
8,Master Data Analyst - Reynella,Adelaide,Accolade Wines Australia,Information & Communication Technology
9,Scientist (Data Analyst),Southern Highlands & Tablelands,"Department of Planning, Industry and Environment",Government & Defence


Duplicates were then removed from the dataset

In [9]:
#removing duplicates if all three columns are the same: 

Job_df2=Job_df.drop_duplicates()

In order to use the Leaflet library, the longitude and latitude co-ordinates needed to be obtained for each job, This will be done using the Google Places API. First, a search string column was added to the dataframe. This Search Engins column will be used as a parameter in the Google Places API. 

In [10]:
#setting up search to use in the API call 
Job_df2["Search Engine"] = Job_df2["Job Company"].astype(str) + " " + Job_df2["Job Location"].astype(str)

/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
Job_df2.head()

,Job Title,Job Location,Job Company,Job Industry,Search Engine
0,Junior Insights Analyst,Sydney,Oneflare Pty Ltd,Sales,Oneflare Pty Ltd Sydney
1,Graduate Data Analyst & Programmer,Gold Coast,Tebbutt Research,Marketing & Communications,Tebbutt Research Gold Coast
2,Data Analyst,West Gippsland & Latrobe Valley,Hays Information Technology,Government & Defence,Hays Information Technology West Gippsland & L...
3,Data Analyst/Manager,Sydney,Finite IT Recruitment Solutions,Information & Communication Technology,Finite IT Recruitment Solutions Sydney
4,Data Analyst,ACT,Hudson - Project Services,Information & Communication Technology,Hudson - Project Services ACT


The below code adds Lat and Lng columns to the dataframe to set up the dataframe further for the Google API 

In [12]:
#adding latitude and longitude columns
Job_df2['Lat'] = ""
Job_df2['Lng']=""

/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


The following code loops through all the search engine column and outputs a Lat and Lng from the Google Places API. Try and Except is used so that if there is an error, the code will not automatically stop. 

In [13]:
for index, row in Job_df2.iterrows():
    
    search_string=row["Search Engine"]
    
    try:
        url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={search_string}&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key={gkey}'
        # Run request
        response = requests.get(url)
        search_geo = response.json()
        
    
        Job_df2.loc[index, 'Lat'] = search_geo["candidates"][0]['geometry']['location']['lat']
        Job_df2.loc[index, "Lng"]=search_geo["candidates"][0]['geometry']['location']['lng']

    
    #in the case of a JSONDecode Error, this ensures the code will keep running
    except (KeyError, IndexError, ValueError):
        Job_df2.loc[index, 'Lat'] = "No Coordinates"
        Job_df2.loc[index, 'Lng'] = "No Coordinates"


/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [14]:
Job_df2.head(15)

,Job Title,Job Location,Job Company,Job Industry,Search Engine,Lat,Lng
0,Junior Insights Analyst,Sydney,Oneflare Pty Ltd,Sales,Oneflare Pty Ltd Sydney,-33.8682,151.204
1,Graduate Data Analyst & Programmer,Gold Coast,Tebbutt Research,Marketing & Communications,Tebbutt Research Gold Coast,-27.9706,153.413
2,Data Analyst,West Gippsland & Latrobe Valley,Hays Information Technology,Government & Defence,Hays Information Technology West Gippsland & L...,-38.1629,145.934
3,Data Analyst/Manager,Sydney,Finite IT Recruitment Solutions,Information & Communication Technology,Finite IT Recruitment Solutions Sydney,-33.8667,151.208
4,Data Analyst,ACT,Hudson - Project Services,Information & Communication Technology,Hudson - Project Services ACT,-35.2759,149.128
5,Graduate Estate Data Analyst,Melbourne,Ventia,Construction,Ventia Melbourne,-37.8575,145.113
6,Data Analyst,Sydney,Teachers Mutual Bank,Information & Communication Technology,Teachers Mutual Bank Sydney,-33.8675,151.206
7,Customer Insights Data Analyst,Brisbane,eraDigital Services,Marketing & Communications,eraDigital Services Brisbane,No Coordinates,No Coordinates
8,Master Data Analyst - Reynella,Adelaide,Accolade Wines Australia,Information & Communication Technology,Accolade Wines Australia Adelaide,-35.0943,138.543
9,Scientist (Data Analyst),Southern Highlands & Tablelands,"Department of Planning, Industry and Environment",Government & Defence,"Department of Planning, Industry and Environme...",-34.4249,150.899


In order to double check, the entries where lat and lng coordinates were not found were just displayed. 

In [15]:
#check the ones that have no co-ordinates
Job_df2.loc[Job_df2['Lat'] == "No Coordinates"]

,Job Title,Job Location,Job Company,Job Industry,Search Engine,Lat,Lng
7,Customer Insights Data Analyst,Brisbane,eraDigital Services,Marketing & Communications,eraDigital Services Brisbane,No Coordinates,No Coordinates
23,Data Analyst,West Gippsland and Latrobe Valley,Hays Information Technology,Government & Defence,Hays Information Technology West Gippsland and...,No Coordinates,No Coordinates
26,Data / Business Analyst,Melbourne,GMAS Pty Limited,Information & Communication Technology,GMAS Pty Limited Melbourne,No Coordinates,No Coordinates
44,Senior Data Analyst,ACT,Adaps IT,Information & Communication Technology,Adaps IT ACT,No Coordinates,No Coordinates
58,APS 6 Policy/Data Analyst,ACT,Hudson - Policy and Program,Government & Defence,Hudson - Policy and Program ACT,No Coordinates,No Coordinates
109,Experience Report Developer,Darwin,XIPNOS Pty Ltd,Information & Communication Technology,XIPNOS Pty Ltd Darwin,No Coordinates,No Coordinates
116,Data Analyst,"Newcastle, Maitland & Hunter",Karlka Recruiting Group,Government & Defence,"Karlka Recruiting Group Newcastle, Maitland & ...",No Coordinates,No Coordinates
139,Data Analyst,Sunshine Coast,NOVA Partners,Information & Communication Technology,NOVA Partners Sunshine Coast,No Coordinates,No Coordinates
150,Data Analyst,ACT,First Grade Recruitment,Information & Communication Technology,First Grade Recruitment ACT,No Coordinates,No Coordinates
153,Logistics and Operations Analyst,Melbourne,POPTECH Consulting,"Manufacturing, Transport & Logistics",POPTECH Consulting Melbourne,No Coordinates,No Coordinates


In order to data-clean, all Jobs where no coordinates were found was then deleted from the dataframe and it was exported as a csv

In [16]:
#Then delete the ones with no co-ordinates
final_Job_df=Job_df2.loc[Job_df2['Lat'] != "No Coordinates"]

In [17]:
#Finally export it to a csv
final_Job_df.to_csv("data/Final_job_df.csv")

In [18]:
df2=pd.read_csv("data/Final_job_df.csv")

In [19]:
#looking at al lthe unique values and mapping them to a state 
df2["Job Location"].unique()


array(['Sydney', 'Gold Coast', 'West Gippsland & Latrobe Valley', 'ACT',
       'Melbourne', 'Adelaide', 'Southern Highlands & Tablelands',
       'Perth', 'Northern QLD', 'Brisbane', 'South West Coast VIC',
       'Bunbury & South West', 'Newcastle, Maitland & Hunter',
       'Wollongong, Illawarra & South Coast', 'Hobart',
       'Cairns & Far North', 'Launceston & North East',
       'Ballarat & Central Highlands', 'Blue Mountains & Central West'],
      dtype=object)

In [20]:
#replacing with states for the location values 
# create ChainMap
L1 = ['Adelaide']
d1 = dict.fromkeys(L1, 'SA')

L2 = ['Gold Coast', "Brisbane", 'Northern QLD', 'Cairns & Far North']
d2 = dict.fromkeys(L2, 'QLD')

L3 = ['Sydney', "Southern Highlands & Tablelands", 'Newcastle, Maitland & Hunter', 'Wollongong, Illawarra & South Coast', 'Blue Mountains & Central West']
d3 = dict.fromkeys(L3, 'NSW')

L4 = ['Melbourne', "South West Coast VIC"]
d4 = dict.fromkeys(L4, 'VIC')

L5 = ['Bunbury & South West', "Perth"]
d5 = dict.fromkeys(L5, 'WA')

L6 = ['Hobart', "Launceston & North East"]
d6 = dict.fromkeys(L6, 'TAS')

L7 = ['Alice Springs & Central Australia']
d7 = dict.fromkeys(L7, 'NT')

L8 = ['ACT']
d8 = dict.fromkeys(L8, 'ACT')


d = {**d1, **d2, **d3, **d4, **d5, **d6, **d7, **d8}
print (d)



{'Adelaide': 'SA', 'Gold Coast': 'QLD', 'Brisbane': 'QLD', 'Northern QLD': 'QLD', 'Cairns & Far North': 'QLD', 'Sydney': 'NSW', 'Southern Highlands & Tablelands': 'NSW', 'Newcastle, Maitland & Hunter': 'NSW', 'Wollongong, Illawarra & South Coast': 'NSW', 'Blue Mountains & Central West': 'NSW', 'Melbourne': 'VIC', 'South West Coast VIC': 'VIC', 'Bunbury & South West': 'WA', 'Perth': 'WA', 'Hobart': 'TAS', 'Launceston & North East': 'TAS', 'Alice Springs & Central Australia': 'NT', 'ACT': 'ACT'}


In [21]:
df2['State'] = df2['Job Location'].map(d)

In [22]:
df2.head(15)

,Unnamed: 0,Job Title,Job Location,Job Company,Job Industry,Search Engine,Lat,Lng,State
0,0,Junior Insights Analyst,Sydney,Oneflare Pty Ltd,Sales,Oneflare Pty Ltd Sydney,-33.868220,151.203777,NSW
1,1,Graduate Data Analyst & Programmer,Gold Coast,Tebbutt Research,Marketing & Communications,Tebbutt Research Gold Coast,-27.970591,153.413115,QLD
2,2,Data Analyst,West Gippsland & Latrobe Valley,Hays Information Technology,Government & Defence,Hays Information Technology West Gippsland & L...,-38.162890,145.933707,NaN
3,3,Data Analyst/Manager,Sydney,Finite IT Recruitment Solutions,Information & Communication Technology,Finite IT Recruitment Solutions Sydney,-33.866704,151.208325,NSW
4,4,Data Analyst,ACT,Hudson - Project Services,Information & Communication Technology,Hudson - Project Services ACT,-35.275898,149.127980,ACT
5,5,Graduate Estate Data Analyst,Melbourne,Ventia,Construction,Ventia Melbourne,-37.857500,145.113248,VIC
6,6,Data Analyst,Sydney,Teachers Mutual Bank,Information & Communication Technology,Teachers Mutual Bank Sydney,-33.867538,151.205714,NSW
7,8,Master Data Analyst - Reynella,Adelaide,Accolade Wines Australia,Information & Communication Technology,Accolade Wines Australia Adelaide,-35.094292,138.543356,SA
8,9,Scientist (Data Analyst),Southern Highlands & Tablelands,"Department of Planning, Industry and Environment",Government & Defence,"Department of Planning, Industry and Environme...",-34.424912,150.898812,NSW
9,10,Scientist (Data Analyst),Southern Highlands & Tablelands,"Department of Planning, Industry and Environment",Science & Technology,"Department of Planning, Industry and Environme...",-34.424912,150.898812,NSW


In [23]:
#count by location the gauge chart 
#Location in NSW
#by sub-category for each state

In [24]:
df2=df2.rename(columns={"Job Industry": "Industry"})

df2["Industry"].unique()


array(['Sales', 'Marketing & Communications', 'Government & Defence',
       'Information & Communication Technology', 'Construction',
       'Science & Technology', 'Banking & Financial Services',
       'Insurance & Superannuation', 'Healthcare & Medical',
       'Administration & Office Support', 'Accounting',
       'Education & Training', 'Consulting & Strategy',
       'Human Resources & Recruitment', 'Real Estate & Property',
       'Hospitality & Tourism', 'Manufacturing, Transport & Logistics',
       'Community Services & Development', 'Engineering',
       'Mining, Resources & Energy'], dtype=object)

In [25]:
Industry2=df2.Industry.map({'Sales' : "Sales", 'Marketing & Communications': 'Marketing', 'Information & Communication Technology' : "ICT",
                 'Government & Defence' : "Govt", 'Banking & Financial Services': "Banking",'Science & Technology': "Science",'Consulting & Strategy' : "Consulting", 
                  "Insurance & Superannuation" : "Insurance", "Administration & Office Support" : "Admin", "Accounting": "Accounting", 
                 "Healthcare & Medical":"Healthcare", "Education & Training" : "Educ", "Human Resources & Recruitment": "HR", "Real Estate & Property": "Real Estate", "Manufacturing, Transport & Logistics":"Transport", "Hospitality & Tourism": "Hospitality", 
                 "Community Services & Development":"Community", "Engineering":"Engineering", "Mining, Resources & Energy":"Mining" })

In [26]:
df2['Job Industry 2'] = Industry2

grouped_state = df2.groupby('State')
grouped_state_1=grouped_state["Job Title"].count()

grouped_state_1
list1=[]



In [27]:
#dropping all NA values
df2=df2.dropna()

In [28]:
grouped_count_state=pd.DataFrame(grouped_state_1)
grouped_state_1=grouped_state_1.to_dict()
list1.append(grouped_state_1)


In [29]:
print(list1)

[{'ACT': 34, 'NSW': 77, 'QLD': 18, 'SA': 8, 'TAS': 4, 'VIC': 37, 'WA': 9}]


In [30]:
grouped_state_2 = df2.groupby(['State','Job Industry 2'])


In [31]:
grouped_state_industry=grouped_state_2["Job Title"].count()

In [32]:
grouped_state_industry=pd.DataFrame(grouped_state_industry)

In [33]:
from collections import defaultdict
results = defaultdict(lambda: defaultdict(dict))

for index, value in grouped_state_industry.itertuples():
    for i, key in enumerate(index):
        if i == 0:
            nested = results[key]
        elif i == len(index) - 1:
            nested[key] = value
        else:
            nested = nested[key]


 

In [34]:
list1.append(results)

In [35]:
grouped_state_3 = df2.groupby(['State','Job Location'])
grouped_state_location=grouped_state_3["Job Title"].count()
grouped_state_location=pd.DataFrame(grouped_state_location)

In [36]:
from collections import defaultdict
results2 = defaultdict(lambda: defaultdict(dict))

for index, value in grouped_state_location.itertuples():
    for i, key in enumerate(index):
        if i == 0:
            nested = results2[key]
        elif i == len(index) - 1:
            nested[key] = value
        else:
            nested = nested[key]

In [37]:

list1.append(results2)


In [38]:

with open('data/job_data.json', 'w') as fp:
    json.dump(list1, fp)